In [87]:
my=[[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0]]

In [88]:
height = len(my) # first dimension
width = 0 # second dimension
for list_ in my:
    if width ==0 :
        width = len(list_)
    for i,item in enumerate(list_):
        if item == 1:
          list_[i] = -1  
        elif item == 0:
          list_[i] = float('inf')
my

[[inf, inf, inf, inf, inf, inf],
 [-1, -1, -1, -1, -1, inf],
 [inf, inf, inf, inf, inf, inf],
 [inf, -1, -1, -1, -1, -1],
 [inf, -1, -1, -1, -1, -1],
 [inf, inf, inf, inf, inf, inf]]

In [89]:
width, height

(6, 6)

In [90]:
def check_point(i,j,height, width):
    # points should not be outside the bounds of the grid
    return 0 <= i < height and 0<= j < width
def get_nearby_points(i,j,height, width):
    points = []
    if check_point(i+1,j,height,width):
        points.append((i+1,j))
    if check_point(i-1,j,height,width):
        points.append((i-1,j))
    if check_point(i, j+1, height,width):
        points.append((i, j+1))
    if check_point(i, j-1, height, width):
        points.append((i, j-1))  
    return points

In [107]:
def update(array, update_point):
    while len(update_point) > 0:
        i,j=update_point.pop()
        for from_i, from_j in get_nearby_points(i,j,height=height, width=width):
            if array[from_i][from_j] == -1:
                continue
            if my[from_i][from_j] + 1 < array[i][j]:
                array[i][j] = array[from_i][from_j] + 1
                update_point.extend(get_nearby_points(i,j,height=height, width=width))
    
                

In [108]:
my[height-1][width-1] = 1
update_points=get_nearby_points(height-1,width-1,height,width)
update(my, update_points)
my

[[21, 20, 19, 18, 17, 16],
 [-1, -1, -1, -1, -1, 15],
 [9, 10, 11, 12, 13, 14],
 [8, -1, -1, -1, -1, -1],
 [7, -1, -1, -1, -1, -1],
 [6, 5, 4, 3, 2, 1]]

In [109]:
my[0][0]

21

In [110]:
min_array_value = float('inf')
min_array = None
for i,list_ in enumerate(my):
    for j,item in enumerate(list_):
        if item == -1:
            copy = [[y for y in x] for x in my]
            min_value = float('inf')
            for i_,j_ in get_nearby_points(i,j,height,width):
                if my[i_][j_] == -1:
                    continue
                min_value = min(copy[i_][j_] + 1, min_value)
            copy[i][j] = min_value
            update_points=get_nearby_points(i,j,height,width)
            update(copy, update_points)
            if copy[0][0]< min_array_value:
                min_array_value = copy[0][0]
                min_array = copy

In [111]:
min_array_value,min_array

(11,
 [[11, 20, 19, 18, 17, 16],
  [10, -1, -1, -1, -1, 15],
  [11, 10, 11, 12, 13, 14],
  [8, -1, -1, -1, -1, -1],
  [7, -1, -1, -1, -1, -1],
  [6, 5, 4, 3, 2, 1]])

In [37]:
class Maze:
    def __init__(self, array):
        self.array = array
        self.height = len(array) # first dimension
        self.width = len(array[0]) # second dimension
        
    def preprocess_array(self):
        for list_ in self.array:
            for i,item in enumerate(list_):
                if item == 1:
                    list_[i] = -1  
                elif item == 0:
                    list_[i] = float('inf')

    def _check_point(self,i,j):
        # points should not be outside the bounds of the grid
        return 0 <= i < self.height and 0 <= j < self.width \
            and self.array[i][j] != -1
            
    def _get_nearby_points(self,i,j):
        points = []
        if self._check_point(i+1,j):
            points.append((i+1,j))
        if self._check_point(i-1,j):
            points.append((i-1,j))
        if self._check_point(i, j+1):
            points.append((i, j+1))
        if self._check_point(i, j-1):
            points.append((i, j-1))  
        return points
    
    def _update_single_point(self, i,j):
        """
        This function updates a single point value based on its neighbours
        This is useful for removing a wall
        """
        min_value = float('inf')
        for i_,j_ in self._get_nearby_points(i,j):
            min_value = min(self.array[i_][j_] + 1, min_value)
        return min_value
    
    def _update(self, update_queue):
        while len(update_queue) > 0:
            i,j=update_queue.pop()
            for from_i, from_j in self._get_nearby_points(i,j):
                if self.array[from_i][from_j] + 1 < self.array[i][j]:
                    self.array[i][j] = self.array[from_i][from_j] + 1
                    update_queue.extend(self._get_nearby_points(i,j))
        
    
    def _set_value(self,i,j,value):
        if value is None:
            value = self._update_single_point(i,j)
        self.array[i][j] = value
        update_points=self._get_nearby_points(i,j)
        self._update(update_points)
    
    def analyze_without_replacement(self):
        self._set_value(self.height-1, self.width-1, 1)
        return self.array
    
    def analyze_with_replacement(self):
        min_array_value = self.array[0][0]
        min_array = self.array
        for i,list_ in enumerate(self.array):
            for j,item in enumerate(list_):
                if item == -1:
                    copy = [[y for y in x] for x in self.array]
                    new_maze=Maze(copy)
                    new_maze._set_value(i,j,None)
                    if new_maze.array[0][0] < min_array_value:
                        min_array_value = new_maze.array[0][0] 
                        min_array = new_maze.array
                        
        return min_array_value, min_array

In [38]:
my_maze=[[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0]]
maze=Maze(my_maze)
maze.preprocess_array()
maze.analyze_without_replacement()
maze.analyze_with_replacement()

(11,
 [[11, 12, 13, 14, 15, 16],
  [10, -1, -1, -1, -1, 15],
  [9, 10, 11, 12, 13, 14],
  [8, -1, -1, -1, -1, -1],
  [7, -1, -1, -1, -1, -1],
  [6, 5, 4, 3, 2, 1]])

In [39]:
my_maze=[[0, 1, 1, 0], [0, 0, 0, 1], [1, 1, 0, 0], [1, 1, 1, 0]]
maze=Maze(my_maze)
maze.preprocess_array()
maze.analyze_without_replacement()
maze.analyze_with_replacement()

(7, [[7, -1, -1, inf], [6, 5, 4, -1], [-1, -1, 3, 2], [-1, -1, -1, 1]])

[[7, 6, -1, inf], [6, 5, 4, -1], [-1, -1, 3, 2], [-1, -1, -1, 1]]